In [1]:
from time import process_time
import numpy as np
import math

with open('data.txt', 'r') as data:
    data = data.read().splitlines()
data = np.array([tuple(line.split(',')) for line in data])
        
iris_data = np.zeros((150,4))

for row in range(150):
    for col in range(4):
        iris_data[row,col] = float(data[row,col])
iris_data = iris_data.T

In [2]:
setosa_training = iris_data[0:4,0:40]
setosa_testing = iris_data[0:4,40:50]

versi_training = iris_data[0:4,50:90]
versi_testing = iris_data[0:4,90:100]

virginica_training = iris_data[0:4,100:140]
virginica_testing = iris_data[0:4,140:150]

training_data = setosa_training
training_data = np.concatenate((training_data,versi_training),axis = 1)
training_data = np.concatenate((training_data,virginica_training),axis = 1)

testing_data = setosa_testing
testing_data = np.concatenate((testing_data,versi_testing),axis = 1)
testing_data = np.concatenate((testing_data,virginica_testing),axis = 1)

In [3]:
setosa_mu = np.array([np.sum(setosa_training, axis = 1)]).T/40
versi_mu = np.array([np.sum(versi_training, axis = 1)]).T/40
virginica_mu = np.array([np.sum(virginica_training, axis = 1)]).T/40

print("Setosa Mu:");
print(setosa_mu);
print("Versi Mu:");
print(versi_mu);
print("Virginica Mu:")
print(virginica_mu);

Setosa Mu:
[[5.0375]
 [3.44  ]
 [1.4625]
 [0.2325]]
Versi Mu:
[[6.01  ]
 [2.78  ]
 [4.3175]
 [1.35  ]]
Virginica Mu:
[[6.6225]
 [2.96  ]
 [5.6075]
 [1.99  ]]


In [4]:
setosa_E = np.matmul((setosa_training-setosa_mu),(setosa_training-setosa_mu).T)/40
versi_E = np.matmul((versi_training-versi_mu),(versi_training-versi_mu).T)/40
virginica_E = np.matmul((virginica_training-virginica_mu),(virginica_training-virginica_mu).T)/40
print("Setosa Sigma")
print(setosa_E)
print("Versi Sigma")
print(versi_E)
print("Virginica Sigma")
print(virginica_E)

LDA_E = (setosa_E + versi_E + virginica_E)/3
print("LDA Sigma")
print(LDA_E)

Setosa Sigma
[[0.12784375 0.0965     0.01265625 0.01328125]
 [0.0965     0.1294     0.002      0.0142    ]
 [0.01265625 0.002      0.02884375 0.00446875]
 [0.01328125 0.0142     0.00446875 0.00969375]]
Versi Sigma
[[0.2669     0.08445    0.167825   0.051     ]
 [0.08445    0.1081     0.07885    0.04425   ]
 [0.167825   0.07885    0.19844375 0.071875  ]
 [0.051      0.04425    0.071875   0.042     ]]
Virginica Sigma
[[0.45624375 0.10765    0.34883125 0.049975  ]
 [0.10765    0.1104     0.07905    0.0451    ]
 [0.34883125 0.07905    0.33669375 0.057825  ]
 [0.049975   0.0451     0.057825   0.0724    ]]
LDA Sigma
[[0.2836625  0.0962     0.1764375  0.03808542]
 [0.0962     0.11596667 0.0533     0.03451667]
 [0.1764375  0.0533     0.18799375 0.04472292]
 [0.03808542 0.03451667 0.04472292 0.04136458]]


In [5]:
def gaussian(E, data, mu, i, k):
    return (1 / ((2 * math.pi)**2 * np.linalg.det(E))**(1 / 2)) * math.exp((-1/2) * np.matmul(np.matmul(data[0:k, i] - mu.T, np.linalg.inv(E)), (data[0:k, i] - mu.T).T))

In [6]:
def QDA_training_population(setosa_E,versi_E,virginica_E, data, setosa_mu, versi_mu, virginica_mu, k):
    QDA_training_classification = []
    for i in range(120):
        prob1 = gaussian(setosa_E, data, setosa_mu, i, k)
        prob2 = gaussian(versi_E, data, versi_mu, i, k)
        prob3 = gaussian(virginica_E, data, virginica_mu, i, k)
        if max(prob1,prob2,prob3) == prob1:
            QDA_training_classification.append('Iris-setosa')
        if max(prob1,prob2,prob3) == prob2:
            QDA_training_classification.append('Iris-versicolor')
        if max(prob1,prob2,prob3) == prob3:
            QDA_training_classification.append('Iris-virginica')
    print("QDA_training_classification")
    print(QDA_training_classification)
    return(QDA_training_classification)

def QDA_testing_population(setosa_E,versi_E,virginica_E, data, setosa_mu, versi_mu, virginica_mu, k):
    QDA_testing_classification = []
    for i in range(30):
        prob1 = gaussian(setosa_E, data, setosa_mu, i, k)
        prob2 = gaussian(versi_E, data, versi_mu, i, k)
        prob3 = gaussian(virginica_E, data, virginica_mu, i, k)
        if max(prob1,prob2,prob3) == prob1:
            QDA_testing_classification.append('Iris-setosa')
        if max(prob1,prob2,prob3) == prob2:
            QDA_testing_classification.append('Iris-versicolor')
        if max(prob1,prob2,prob3) == prob3:
            QDA_testing_classification.append('Iris-virginica')
    print("QDA_testing_classification")
    print(QDA_testing_classification)
    return(QDA_testing_classification)

In [7]:
def LDA_training_population(LDA_E, data, setosa_mu, versi_mu, virginica_mu, k):
    LDA_training_classification = []
    for i in range(120):
        prob1 = gaussian(LDA_E, data, setosa_mu, i, k)
        prob2 = gaussian(LDA_E, data, versi_mu, i, k)
        prob3 = gaussian(LDA_E, data, virginica_mu, i, k)
        if max(prob1,prob2,prob3) == prob1:
            LDA_training_classification.append('Iris-setosa')
        if max(prob1,prob2,prob3) == prob2:
            LDA_training_classification.append('Iris-versicolor')
        if max(prob1,prob2,prob3) == prob3:
            LDA_training_classification.append('Iris-virginica')
    print("LDA_training_classification")
    print(LDA_training_classification)
    return(LDA_training_classification)

def LDA_testing_population(LDA_E, data, setosa_mu, versi_mu, virginica_mu, k):
    LDA_testing_classification = []
    for i in range(30):
        prob1 = gaussian(LDA_E, data, setosa_mu, i, k)
        prob2 = gaussian(LDA_E, data, versi_mu, i, k)
        prob3 = gaussian(LDA_E, data, virginica_mu, i, k)
        if max(prob1,prob2,prob3) == prob1:
            LDA_testing_classification.append('Iris-setosa')
        if max(prob1,prob2,prob3) == prob2:
            LDA_testing_classification.append('Iris-versicolor')
        if max(prob1,prob2,prob3) == prob3:
            LDA_testing_classification.append('Iris-virginica')
    print("LDA_testing_classification")
    print(LDA_testing_classification)
    return(LDA_testing_classification)

In [8]:
def training_error_count(data):
    errorcount = 0
    for i in range(40):
        if data[i] != 'Iris-setosa':
            print("%s at element %s should have been Iris-setosa" %(data[i], i))
            errorcount = errorcount + 1
    for i in range(40,80):
        if data[i] != 'Iris-versicolor':
            print("%s at element %s should have been Iris-versicolor" %(data[i], i))
            errorcount = errorcount + 1
    for i in range(80,120):
        if data[i] != 'Iris-virginica':
            print("%s at element %s should have been Iris-virginica" %(data[i], i))
            errorcount = errorcount + 1
    return errorcount

def testing_error_count(data):    
    errorcount = 0
    for i in range(10):
        if data[i] != 'Iris-setosa':
            print("%s at element %s should have been Iris-setosa" %(data[i], i))
            errorcount = errorcount + 1
    for i in range(10,20):
        if data[i] != 'Iris-versicolor':
            print("%s at element %s should have been Iris-versicolor" %(data[i], i))
            errorcount = errorcount + 1
    for i in range(20,30):
        if data[i] != 'Iris-virginica':
            print("%s at element %s should have been Iris-virginica" %(data[i], i))
            errorcount = errorcount + 1
    return errorcount


In [9]:
start = process_time()
QDA_all_training = []
QDA_all_training = QDA_training_population(setosa_E,versi_E,virginica_E, training_data, setosa_mu, versi_mu, virginica_mu, 4)
end = process_time()
total_time_reg = end - start
print("\nTotal Time:")
print(total_time_reg)   

QDA_training_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-vers

In [10]:
print("Error Rate for QDA Training Data: %s" %(training_error_count(QDA_all_training)/120))


Iris-virginica at element 60 should have been Iris-versicolor
Iris-virginica at element 73 should have been Iris-versicolor
Error Rate for QDA Training Data: 0.016666666666666666


In [11]:
QDA_all_testing = []
QDA_all_testing = QDA_testing_population(setosa_E,versi_E,virginica_E, testing_data, setosa_mu, versi_mu, virginica_mu, 4)


QDA_testing_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica']


In [12]:
print("Error Rate for QDA Testing Data: %s" %(testing_error_count(QDA_all_testing)/30))



Error Rate for QDA Testing Data: 0.0


In [13]:
LDA_all_training = []
LDA_all_training = LDA_training_population(LDA_E, training_data, setosa_mu, versi_mu, virginica_mu, 4)


LDA_training_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-vers

In [14]:
print("Error Rate for LDA Training Data: %s" %(training_error_count(LDA_all_training)/120))


Iris-virginica at element 60 should have been Iris-versicolor
Iris-virginica at element 73 should have been Iris-versicolor
Iris-versicolor at element 113 should have been Iris-virginica
Error Rate for LDA Training Data: 0.025


In [15]:
LDA_all_testing = []
LDA_all_testing = LDA_testing_population(LDA_E, testing_data, setosa_mu, versi_mu, virginica_mu, 4)


LDA_testing_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica']


In [16]:
print("Error Rate for LDA Testing Data: %s" %(testing_error_count(LDA_all_testing)/30))



Error Rate for LDA Testing Data: 0.0


# Removing Sepal Length

In [17]:
training_data_noSl = training_data[1:4]
#print("Training No SL")
#print(training_data_noSl)

testing_data_noSl = testing_data[1:4]
#print("Testing No SL")
#print(testing_data_noSl)

In [18]:
setosa_training_noSl = setosa_training[1:4]
setosa_testing_noSl = setosa_testing[1:4]

versi_training_noSl = versi_training[1:4]
versi_testing_noSl = versi_testing[1:4]

virginica_training_noSl = virginica_training[1:4]
virginica_testing_noSl = virginica_training[1:4]

In [19]:
setosa_mu_noSl = np.array([np.sum(setosa_training_noSl, axis = 1)]).T/40
versi_mu_noSl = np.array([np.sum(versi_training_noSl, axis = 1)]).T/40
virginica_mu_noSl = np.array([np.sum(virginica_training_noSl, axis = 1)]).T/40

print("Setosa Mu:");
print(setosa_mu_noSl);
print("Versi Mu:");
print(versi_mu_noSl);
print("Virginica Mu:")
print(virginica_mu_noSl);

Setosa Mu:
[[3.44  ]
 [1.4625]
 [0.2325]]
Versi Mu:
[[2.78  ]
 [4.3175]
 [1.35  ]]
Virginica Mu:
[[2.96  ]
 [5.6075]
 [1.99  ]]


In [20]:
setosa_E_noSl = np.matmul((setosa_training_noSl-setosa_mu_noSl),(setosa_training_noSl-setosa_mu_noSl).T)/40
versi_E_noSl = np.matmul((versi_training_noSl-versi_mu_noSl),(versi_training_noSl-versi_mu_noSl).T)/40
virginica_E_noSl = np.matmul((virginica_training_noSl-virginica_mu_noSl),(virginica_training_noSl-virginica_mu_noSl).T)/40
print("Setosa Sigma")
print(setosa_E_noSl)
print("Versi Sigma")
print(versi_E_noSl)
print("Virginica Sigma")
print(virginica_E_noSl)

LDA_E_noSl = (setosa_E_noSl + versi_E_noSl + virginica_E_noSl)/3
print("LDA Sigma")
print(LDA_E_noSl)

Setosa Sigma
[[0.1294     0.002      0.0142    ]
 [0.002      0.02884375 0.00446875]
 [0.0142     0.00446875 0.00969375]]
Versi Sigma
[[0.1081     0.07885    0.04425   ]
 [0.07885    0.19844375 0.071875  ]
 [0.04425    0.071875   0.042     ]]
Virginica Sigma
[[0.1104     0.07905    0.0451    ]
 [0.07905    0.33669375 0.057825  ]
 [0.0451     0.057825   0.0724    ]]
LDA Sigma
[[0.11596667 0.0533     0.03451667]
 [0.0533     0.18799375 0.04472292]
 [0.03451667 0.04472292 0.04136458]]


In [21]:
QDA_noSl_training = []
QDA_noSl_training = QDA_training_population(setosa_E_noSl,versi_E_noSl,virginica_E_noSl, training_data_noSl, setosa_mu_noSl, versi_mu_noSl, virginica_mu_noSl, 3)


QDA_training_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-versi

In [22]:
print("Error Rate for QDA Training Data No Sl: %s" %(training_error_count(QDA_noSl_training)/120))


Iris-virginica at element 58 should have been Iris-versicolor
Iris-virginica at element 73 should have been Iris-versicolor
Iris-versicolor at element 113 should have been Iris-virginica
Error Rate for QDA Training Data No Sl: 0.025


In [23]:
QDA_noSl_testing = []
QDA_noSl_testing = QDA_testing_population(setosa_E_noSl,versi_E_noSl,virginica_E_noSl, testing_data_noSl, setosa_mu_noSl, versi_mu_noSl, virginica_mu_noSl, 3)


QDA_testing_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica']


In [24]:
print("Error Rate for QDA Testing Data No Sl: %s" %(testing_error_count(QDA_noSl_testing)/30))



Error Rate for QDA Testing Data No Sl: 0.0


In [25]:
LDA_noSl_training = []
LDA_noSl_training = LDA_training_population(LDA_E_noSl, training_data_noSl, setosa_mu_noSl, versi_mu_noSl, virginica_mu_noSl, 3)


LDA_training_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-vers

In [26]:
print("Error Rate for LDA Training Data No Sl: %s" %(training_error_count(LDA_noSl_training)/120))


Iris-virginica at element 73 should have been Iris-versicolor
Iris-versicolor at element 113 should have been Iris-virginica
Error Rate for LDA Training Data No Sl: 0.016666666666666666


In [27]:
LDA_noSl_testing = []
LDA_noSl_testing = LDA_testing_population(LDA_E_noSl, testing_data_noSl, setosa_mu_noSl, versi_mu_noSl, virginica_mu_noSl, 3)


LDA_testing_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica']


In [28]:
print("Error Rate for LDA Testing Data No Sl: %s" %(testing_error_count(LDA_noSl_testing)/30))



Error Rate for LDA Testing Data No Sl: 0.0


# Removing Sepal Width

In [29]:
training_data_noSw = np.concatenate(([training_data[0]],training_data[2:4]), axis = 0)

testing_data_noSw = np.concatenate(([testing_data[0]],testing_data[2:4]), axis = 0)

In [30]:
setosa_training_noSw = np.concatenate(([setosa_training[0]],setosa_training[2:4]), axis = 0)
setosa_testing_noSw = np.concatenate(([setosa_testing[0]],setosa_testing[2:4]), axis = 0)

versi_training_noSw = np.concatenate(([versi_training[0]],versi_training[2:4]), axis = 0)
versi_testing_noSw = np.concatenate(([versi_testing[0]],versi_testing[2:4]), axis = 0)

virginica_training_noSw = np.concatenate(([virginica_training[0]],virginica_training[2:4]), axis = 0)
virginica_testing_noSw = np.concatenate(([virginica_testing[0]],virginica_testing[2:4]), axis = 0)

In [31]:
setosa_mu_noSw = np.array([np.sum(setosa_training_noSw, axis = 1)]).T/40
versi_mu_noSw = np.array([np.sum(versi_training_noSw, axis = 1)]).T/40
virginica_mu_noSw = np.array([np.sum(virginica_training_noSw, axis = 1)]).T/40

print("Setosa Mu:");
print(setosa_mu_noSw);
print("Versi Mu:");
print(versi_mu_noSw);
print("Virginica Mu:")
print(virginica_mu_noSw);

Setosa Mu:
[[5.0375]
 [1.4625]
 [0.2325]]
Versi Mu:
[[6.01  ]
 [4.3175]
 [1.35  ]]
Virginica Mu:
[[6.6225]
 [5.6075]
 [1.99  ]]


In [32]:
setosa_E_noSw = np.matmul((setosa_training_noSw-setosa_mu_noSw),(setosa_training_noSw-setosa_mu_noSw).T)/40
versi_E_noSw = np.matmul((versi_training_noSw-versi_mu_noSw),(versi_training_noSw-versi_mu_noSw).T)/40
virginica_E_noSw = np.matmul((virginica_training_noSw-virginica_mu_noSw),(virginica_training_noSw-virginica_mu_noSw).T)/40
print("Setosa Sigma")
print(setosa_E_noSw)
print("Versi Sigma")
print(versi_E_noSw)
print("Virginica Sigma")
print(virginica_E_noSw)

LDA_E_noSw = (setosa_E_noSw + versi_E_noSw + virginica_E_noSw)/3
print("LDA Sigma")
print(LDA_E_noSw)

Setosa Sigma
[[0.12784375 0.01265625 0.01328125]
 [0.01265625 0.02884375 0.00446875]
 [0.01328125 0.00446875 0.00969375]]
Versi Sigma
[[0.2669     0.167825   0.051     ]
 [0.167825   0.19844375 0.071875  ]
 [0.051      0.071875   0.042     ]]
Virginica Sigma
[[0.45624375 0.34883125 0.049975  ]
 [0.34883125 0.33669375 0.057825  ]
 [0.049975   0.057825   0.0724    ]]
LDA Sigma
[[0.2836625  0.1764375  0.03808542]
 [0.1764375  0.18799375 0.04472292]
 [0.03808542 0.04472292 0.04136458]]


In [33]:
QDA_noSw_training = []
QDA_noSw_training = QDA_training_population(setosa_E_noSw,versi_E_noSw,virginica_E_noSw, training_data_noSw, setosa_mu_noSw, versi_mu_noSw, virginica_mu_noSw, 3)


QDA_training_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-vers

In [34]:
print("Error Rate for QDA Training Data No Sw: %s" %(training_error_count(QDA_noSw_training)/120))


Iris-virginica at element 60 should have been Iris-versicolor
Iris-virginica at element 73 should have been Iris-versicolor
Error Rate for QDA Training Data No Sw: 0.016666666666666666


In [35]:
QDA_noSw_testing = []
QDA_noSw_testing = QDA_testing_population(setosa_E_noSw,versi_E_noSw,virginica_E_noSw, testing_data_noSw, setosa_mu_noSw, versi_mu_noSw, virginica_mu_noSw, 3)


QDA_testing_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica']


In [36]:
print("Error Rate for QDA Testing Data No Sw: %s" %(testing_error_count(QDA_noSw_testing)/30))



Error Rate for QDA Testing Data No Sw: 0.0


In [37]:
LDA_noSw_training = []
LDA_noSw_training = LDA_training_population(LDA_E_noSw, training_data_noSw, setosa_mu_noSw, versi_mu_noSw, virginica_mu_noSw, 3)


LDA_training_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-vers

In [38]:
print("Error Rate for LDA Training Data No Sw: %s" %(training_error_count(LDA_noSw_training)/120))


Iris-virginica at element 60 should have been Iris-versicolor
Iris-virginica at element 73 should have been Iris-versicolor
Iris-versicolor at element 113 should have been Iris-virginica
Error Rate for LDA Training Data No Sw: 0.025


In [39]:
LDA_noSw_testing = []
LDA_noSw_testing = LDA_testing_population(LDA_E_noSw, testing_data_noSw, setosa_mu_noSw, versi_mu_noSw, virginica_mu_noSw, 3)


LDA_testing_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica']


In [40]:
print("Error Rate for LDA Testing Data No Sw: %s" %(testing_error_count(LDA_noSw_testing)/30))



Error Rate for LDA Testing Data No Sw: 0.0


# Removing Petal Length

In [41]:
training_data_noPl = np.concatenate((training_data[0:2],[training_data[3]]), axis = 0)

testing_data_noPl = np.concatenate((testing_data[0:2],[testing_data[3]]), axis = 0)

In [42]:
setosa_training_noPl = np.concatenate((setosa_training[0:2],[setosa_training[3]]), axis = 0)
setosa_testing_noPl = np.concatenate((setosa_testing[0:2],[setosa_testing[3]]), axis = 0)

versi_training_noPl = np.concatenate((versi_training[0:2],[versi_training[3]]), axis = 0)
versi_testing_noPl = np.concatenate((versi_testing[0:2],[versi_testing[3]]), axis = 0)

virginica_training_noPl = np.concatenate((virginica_training[0:2],[virginica_training[3]]), axis = 0)
virginica_testing_noPl = np.concatenate((virginica_testing[0:2],[virginica_testing[3]]), axis = 0)

In [43]:
setosa_mu_noPl = np.array([np.sum(setosa_training_noPl, axis = 1)]).T/40
versi_mu_noPl = np.array([np.sum(versi_training_noPl, axis = 1)]).T/40
virginica_mu_noPl = np.array([np.sum(virginica_training_noPl, axis = 1)]).T/40

print("Setosa Mu:");
print(setosa_mu_noPl);
print("Versi Mu:");
print(versi_mu_noPl);
print("Virginica Mu:")
print(virginica_mu_noPl);

Setosa Mu:
[[5.0375]
 [3.44  ]
 [0.2325]]
Versi Mu:
[[6.01]
 [2.78]
 [1.35]]
Virginica Mu:
[[6.6225]
 [2.96  ]
 [1.99  ]]


In [44]:
setosa_E_noPl = np.matmul((setosa_training_noPl-setosa_mu_noPl),(setosa_training_noPl-setosa_mu_noPl).T)/40
versi_E_noPl = np.matmul((versi_training_noPl-versi_mu_noPl),(versi_training_noPl-versi_mu_noPl).T)/40
virginica_E_noPl = np.matmul((virginica_training_noPl-virginica_mu_noPl),(virginica_training_noPl-virginica_mu_noPl).T)/40
print("Setosa Sigma")
print(setosa_E_noPl)
print("Versi Sigma")
print(versi_E_noPl)
print("Virginica Sigma")
print(virginica_E_noPl)

LDA_E_noPl = (setosa_E_noPl + versi_E_noPl + virginica_E_noPl)/3
print("LDA Sigma")
print(LDA_E_noPl)

Setosa Sigma
[[0.12784375 0.0965     0.01328125]
 [0.0965     0.1294     0.0142    ]
 [0.01328125 0.0142     0.00969375]]
Versi Sigma
[[0.2669  0.08445 0.051  ]
 [0.08445 0.1081  0.04425]
 [0.051   0.04425 0.042  ]]
Virginica Sigma
[[0.45624375 0.10765    0.049975  ]
 [0.10765    0.1104     0.0451    ]
 [0.049975   0.0451     0.0724    ]]
LDA Sigma
[[0.2836625  0.0962     0.03808542]
 [0.0962     0.11596667 0.03451667]
 [0.03808542 0.03451667 0.04136458]]


In [45]:
QDA_noPl_training = []
QDA_noPl_training = QDA_training_population(setosa_E_noPl,versi_E_noPl,virginica_E_noPl, training_data_noPl, setosa_mu_noPl, versi_mu_noPl, virginica_mu_noPl, 3)


QDA_training_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-versi

In [46]:
print("Error Rate for QDA Training Data No Pl: %s" %(training_error_count(QDA_noPl_training)/120))


Iris-virginica at element 58 should have been Iris-versicolor
Iris-virginica at element 60 should have been Iris-versicolor
Iris-virginica at element 67 should have been Iris-versicolor
Iris-versicolor at element 113 should have been Iris-virginica
Iris-versicolor at element 114 should have been Iris-virginica
Error Rate for QDA Training Data No Pl: 0.041666666666666664


In [47]:
QDA_noPl_testing = []
QDA_noPl_testing = QDA_testing_population(setosa_E_noPl,versi_E_noPl,virginica_E_noPl, testing_data_noPl, setosa_mu_noPl, versi_mu_noPl, virginica_mu_noPl, 3)


QDA_testing_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica']


In [48]:
print("Error Rate for QDA Testing Data No Sw: %s" %(testing_error_count(QDA_noPl_testing)/30))



Error Rate for QDA Testing Data No Sw: 0.0


In [49]:
LDA_noPl_training = []
LDA_noPl_training = LDA_training_population(LDA_E_noPl, training_data_noPl, setosa_mu_noPl, versi_mu_noPl, virginica_mu_noPl, 3)


LDA_training_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-versi

In [50]:
print("Error Rate for LDA Training Data No Sw: %s" %(training_error_count(LDA_noPl_training)/120))


Iris-virginica at element 58 should have been Iris-versicolor
Iris-virginica at element 60 should have been Iris-versicolor
Iris-virginica at element 67 should have been Iris-versicolor
Iris-versicolor at element 99 should have been Iris-virginica
Iris-versicolor at element 109 should have been Iris-virginica
Iris-versicolor at element 113 should have been Iris-virginica
Iris-versicolor at element 114 should have been Iris-virginica
Error Rate for LDA Training Data No Sw: 0.058333333333333334


In [51]:
LDA_noPl_testing = []
LDA_noPl_testing = LDA_testing_population(LDA_E_noPl, testing_data_noPl, setosa_mu_noPl, versi_mu_noPl, virginica_mu_noPl, 3)


LDA_testing_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica']


In [52]:
print("Error Rate for LDA Testing Data No Sw: %s" %(testing_error_count(LDA_noPl_testing)/30))



Error Rate for LDA Testing Data No Sw: 0.0


# Removing Petal Width

In [53]:
training_data_noPw = training_data[0:3]

testing_data_noPw = testing_data[0:3]

In [54]:
setosa_training_noPw = setosa_training[0:3]
setosa_testing_noPw = setosa_testing[0:3]

versi_training_noPw = versi_training[0:3]
versi_testing_noPw = versi_testing[0:3]

virginica_training_noPw = virginica_training[0:3]
virginica_testing_noPw = virginica_training[0:3]

In [55]:
setosa_mu_noPw = np.array([np.sum(setosa_training_noPw, axis = 1)]).T/40
versi_mu_noPw = np.array([np.sum(versi_training_noPw, axis = 1)]).T/40
virginica_mu_noPw = np.array([np.sum(virginica_training_noPw, axis = 1)]).T/40

print("Setosa Mu:");
print(setosa_mu_noPw);
print("Versi Mu:");
print(versi_mu_noPw);
print("Virginica Mu:")
print(virginica_mu_noPw);

Setosa Mu:
[[5.0375]
 [3.44  ]
 [1.4625]]
Versi Mu:
[[6.01  ]
 [2.78  ]
 [4.3175]]
Virginica Mu:
[[6.6225]
 [2.96  ]
 [5.6075]]


In [56]:
setosa_E_noPw = np.matmul((setosa_training_noPw-setosa_mu_noPw),(setosa_training_noPw-setosa_mu_noPw).T)/40
versi_E_noPw = np.matmul((versi_training_noPw-versi_mu_noPw),(versi_training_noPw-versi_mu_noPw).T)/40
virginica_E_noPw = np.matmul((virginica_training_noPw-virginica_mu_noPw),(virginica_training_noPw-virginica_mu_noPw).T)/40
print("Setosa Sigma")
print(setosa_E_noPw)
print("Versi Sigma")
print(versi_E_noPw)
print("Virginica Sigma")
print(virginica_E_noPw)

LDA_E_noPw = (setosa_E_noPw + versi_E_noPw + virginica_E_noPw)/3
print("LDA Sigma")
print(LDA_E_noPw)

Setosa Sigma
[[0.12784375 0.0965     0.01265625]
 [0.0965     0.1294     0.002     ]
 [0.01265625 0.002      0.02884375]]
Versi Sigma
[[0.2669     0.08445    0.167825  ]
 [0.08445    0.1081     0.07885   ]
 [0.167825   0.07885    0.19844375]]
Virginica Sigma
[[0.45624375 0.10765    0.34883125]
 [0.10765    0.1104     0.07905   ]
 [0.34883125 0.07905    0.33669375]]
LDA Sigma
[[0.2836625  0.0962     0.1764375 ]
 [0.0962     0.11596667 0.0533    ]
 [0.1764375  0.0533     0.18799375]]


In [57]:
QDA_noPw_training = []
QDA_noPw_training = QDA_training_population(setosa_E_noPw,versi_E_noPw,virginica_E_noPw, training_data_noPw, setosa_mu_noPw, versi_mu_noPw, virginica_mu_noPw, 3)


QDA_training_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-vers

In [58]:
print("Error Rate for QDA Training Data No Pw: %s" %(training_error_count(QDA_noPw_training)/120))


Iris-virginica at element 60 should have been Iris-versicolor
Iris-virginica at element 73 should have been Iris-versicolor
Iris-virginica at element 74 should have been Iris-versicolor
Iris-versicolor at element 90 should have been Iris-virginica
Iris-versicolor at element 103 should have been Iris-virginica
Iris-versicolor at element 106 should have been Iris-virginica
Iris-versicolor at element 118 should have been Iris-virginica
Error Rate for QDA Training Data No Pw: 0.058333333333333334


In [59]:
QDA_noPw_testing = []
QDA_noPw_testing = QDA_testing_population(setosa_E_noPw,versi_E_noPw,virginica_E_noPw, testing_data_noPw, setosa_mu_noPw, versi_mu_noPw, virginica_mu_noPw, 3)


QDA_testing_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica']


In [60]:
print("Error Rate for QDA Testing Data No Pw: %s" %(testing_error_count(QDA_noPw_testing)/30))



Iris-versicolor at element 21 should have been Iris-virginica
Error Rate for QDA Testing Data No Pw: 0.03333333333333333


In [61]:
LDA_noPw_training = []
LDA_noPw_training = LDA_training_population(LDA_E_noPw, training_data_noPw, setosa_mu_noPw, versi_mu_noPw, virginica_mu_noPw, 3)


LDA_training_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-vers

In [62]:
print("Error Rate for LDA Training Data No Pw: %s" %(training_error_count(LDA_noPw_training)/120))


Iris-virginica at element 60 should have been Iris-versicolor
Iris-virginica at element 73 should have been Iris-versicolor
Iris-versicolor at element 103 should have been Iris-virginica
Iris-versicolor at element 106 should have been Iris-virginica
Iris-versicolor at element 118 should have been Iris-virginica
Error Rate for LDA Training Data No Pw: 0.041666666666666664


In [63]:
LDA_noPw_testing = []
LDA_noPw_testing = LDA_testing_population(LDA_E_noPw, testing_data_noPw, setosa_mu_noPw, versi_mu_noPw, virginica_mu_noPw, 3)


LDA_testing_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica']


In [64]:
print("Error Rate for LDA Testing Data No Pw: %s" %(testing_error_count(LDA_noPw_testing)/30))



Iris-versicolor at element 21 should have been Iris-virginica
Error Rate for LDA Testing Data No Pw: 0.03333333333333333


# Diagonal Matrix

In [65]:
setosa_diagonal_E = np.zeros((4,4))
versi_diagonal_E = np.zeros((4,4))
virginica_diagonal_E = np.zeros((4,4))

for i in range(4):
    setosa_diagonal_E[i,i] = setosa_E[i,i]
    versi_diagonal_E[i,i] = versi_E[i,i]
    virginica_diagonal_E[i,i] = virginica_E[i,i]

print(setosa_diagonal_E)
print(versi_diagonal_E)
print(virginica_diagonal_E)

[[0.12784375 0.         0.         0.        ]
 [0.         0.1294     0.         0.        ]
 [0.         0.         0.02884375 0.        ]
 [0.         0.         0.         0.00969375]]
[[0.2669     0.         0.         0.        ]
 [0.         0.1081     0.         0.        ]
 [0.         0.         0.19844375 0.        ]
 [0.         0.         0.         0.042     ]]
[[0.45624375 0.         0.         0.        ]
 [0.         0.1104     0.         0.        ]
 [0.         0.         0.33669375 0.        ]
 [0.         0.         0.         0.0724    ]]


In [66]:
QDA_diagonal_training = []
QDA_diagonal_training = QDA_training_population(setosa_diagonal_E,versi_diagonal_E,virginica_diagonal_E, training_data, setosa_mu, versi_mu, virginica_mu, 4)


QDA_training_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-vers

In [67]:
print("Error Rate for QDA Training Data Diagonal: %s" %(training_error_count(QDA_diagonal_training)/120))


Iris-virginica at element 60 should have been Iris-versicolor
Iris-virginica at element 67 should have been Iris-versicolor
Iris-versicolor at element 86 should have been Iris-virginica
Iris-versicolor at element 99 should have been Iris-virginica
Iris-versicolor at element 113 should have been Iris-virginica
Error Rate for QDA Training Data Diagonal: 0.041666666666666664


In [68]:
start = process_time()
QDA_diagonal_testing = []
QDA_diagonal_testing = QDA_testing_population(setosa_diagonal_E,versi_diagonal_E,virginica_diagonal_E, testing_data, setosa_mu, versi_mu, virginica_mu, 4)
end = process_time()
total_time_diagonal = end - start
print("\nTotal Time:")
print(total_time_diagonal)   



QDA_testing_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica']

Total Time:
0.003902000000000072


In [69]:
print("Error Rate for QDA Testing Data No Sw: %s" %(testing_error_count(QDA_diagonal_testing)/30))



Error Rate for QDA Testing Data No Sw: 0.0


# Checking if classes are linearly separable

In [70]:
noSetosa_data = versi_training
noSetosa_data = np.concatenate((noSetosa_data,virginica_training),axis = 1)
#print(noSetosa_data)

noVersi_data = setosa_training
noVersi_data = np.concatenate((noVersi_data,virginica_training),axis = 1)
#print(noVersi_data)

noVirginica_data = setosa_training
noVirginica_data = np.concatenate((noVirginica_data,versi_training),axis = 1)
#print(noVirginica_data)

In [71]:
LDA_noSetosa_training = []
for i in range(80):
    prob1 = gaussian(LDA_E, noSetosa_data, versi_mu, i, 4)
    prob2 = gaussian(LDA_E, noSetosa_data, virginica_mu, i, 4)
    if max(prob1,prob2) == prob1:
        LDA_noSetosa_training.append('Iris-versicolor')
    if max(prob1,prob2) == prob2:
        LDA_noSetosa_training.append('Iris-virginica')
print("LDA_noSetosa_classification")
print(LDA_noSetosa_training)

LDA_noSetosa_classification
['Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica

In [72]:
errorcount = 0
for i in range(40):
    if LDA_noSetosa_training[i] != 'Iris-versicolor':
        print("%s at element %s should have been Iris-versicolor" %(LDA_noSetosa_training[i], i))
        errorcount = errorcount + 1
for i in range(40,80):
    if LDA_noSetosa_training[i] != 'Iris-virginica':
        print("%s at element %s should have been Iris-virginica" %(LDA_noSetosa_training[i], i))
        errorcount = errorcount + 1
print("Error Rate: %s" %(errorcount/80))


Iris-virginica at element 20 should have been Iris-versicolor
Iris-virginica at element 33 should have been Iris-versicolor
Iris-versicolor at element 73 should have been Iris-virginica
Error Rate: 0.0375


In [73]:
LDA_noVersi_training = []
for i in range(80):
    prob1 = gaussian(LDA_E, noVersi_data, setosa_mu, i, 4)
    prob2 = gaussian(LDA_E, noVersi_data, virginica_mu, i, 4)
    if max(prob1,prob2) == prob1:
        LDA_noVersi_training.append('Iris-setosa')
    if max(prob1,prob2) == prob2:
        LDA_noVersi_training.append('Iris-virginica')
print("LDA_noVersi_classification")
print(LDA_noVersi_training)

LDA_noVersi_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virgin

In [74]:
errorcount = 0
for i in range(40):
    if LDA_noVersi_training[i] != 'Iris-setosa':
        print("%s at element %s should have been Iris-setosa" %(LDA_noVersi_training[i], i))
        errorcount = errorcount + 1
for i in range(40,80):
    if LDA_noVersi_training[i] != 'Iris-virginica':
        print("%s at element %s should have been Iris-virginica" %(LDA_noVersi_training[i], i))
        errorcount = errorcount + 1
print("Error Rate: %s" %(errorcount/80))


Error Rate: 0.0


In [75]:
LDA_noVirginica_training = []
for i in range(80):
    prob1 = gaussian(LDA_E, noVirginica_data, setosa_mu, i, 4)
    prob2 = gaussian(LDA_E, noVirginica_data, versi_mu, i, 4)
    if max(prob1,prob2) == prob1:
        LDA_noVirginica_training.append('Iris-setosa')
    if max(prob1,prob2) == prob2:
        LDA_noVirginica_training.append('Iris-versicolor')
print("LDA_noVirginica_classification")
print(LDA_noVirginica_training)

LDA_noVirginica_classification
['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-v

In [76]:
errorcount = 0
for i in range(40):
    if LDA_noVirginica_training[i] != 'Iris-setosa':
        print("%s at element %s should have been Iris-setosa" %(LDA_noVirginica_training[i], i))
        errorcount = errorcount + 1
for i in range(40,80):
    if LDA_noVirginica_training[i] != 'Iris-versicolor':
        print("%s at element %s should have been Iris-versicolor" %(LDA_noVirginica_training[i], i))
        errorcount = errorcount + 1
print("Error Rate: %s" %(errorcount/80))


Error Rate: 0.0
